In [1]:
%cd ~/repo/protein-transfer

/home/t-fli/repo/protein-transfer


In [2]:
%load_ext blackcellmagic

In [28]:
import numpy as np
import pandas as pd

In [22]:
pd.read_csv("data/structure/secondary_structure/casp12.csv").target.dtype.kind 

'O'

In [23]:
pd.read_csv("data/structure/secondary_structure/casp12.csv").target[0]

'[2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2]'

In [25]:
df = pd.read_csv("data/annotation/scl/balanced.csv")
df.head()

,sequence,target,set,validation
0,MEVLEEPAPGPGGADAAERRGLRRLLLSGFQEELRALLVLAGPAFL...,Cell membrane,train,NaN
1,MMKTLSSGNCTLNVPAKNSYRMVVLGASRVGKSSIVSRFLNGRFED...,Cell membrane,train,NaN
2,MAKRTFSNLETFLIFLLVMMSAITVALLSLLFITSGTIENHKDLGG...,Cell membrane,train,NaN
3,MGNCQAGHNLHLCLAHHPPLVCATLILLLLGLSGLGLGSFLLTHRT...,Cell membrane,train,NaN
4,MDPSKQGTLNRVENSVYRTAFKLRSVQTLCQLDLMDSFLIQQVLWR...,Cell membrane,train,NaN


In [6]:
df.validation.unique(), df.set.unique()

(array([nan, True], dtype=object), array(['train', 'test'], dtype=object))

In [7]:
df.target.unique()

array(['Cell membrane', 'Cytoplasm', 'Endoplasmic reticulum',
       'Golgi apparatus', 'Lysosome/Vacuole', 'Mitochondrion', 'Nucleus',
       'Peroxisome', 'Plastid', 'Extracellular'], dtype=object)

In [27]:
df.target.nunique()

10

In [36]:
df2 = pd.read_csv("data/structure/secondary_structure/casp12.csv")
len(np.unique(np.array(df2["target"][0][1:-1].split(", "))))
# df2["target"] = df2["target"].apply(lambda x: np.array(x[1:-1].split(", ")))

3

In [29]:
"""A script with model training and testing details assuming"""

from __future__ import annotations

import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

from sklearn.metrics import mean_squared_error, log_loss, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder

from scr.params.sys import RAND_SEED, DEVICE

# seed everything
random.seed(RAND_SEED)
np.random.seed(RAND_SEED)
torch.manual_seed(RAND_SEED)
torch.cuda.manual_seed(RAND_SEED)
torch.cuda.manual_seed_all(RAND_SEED)
torch.backends.cudnn.deterministic = True


def get_x_y(
    model, device, batch, embed_layer: int,
):
    """
    A function process x and y from the loader

    Args:
    -

    Returns:
    - x
    - y
    """

    # for each batch: y, sequence, mut_name, mut_numb, [layer0, ...]
    x = batch[4][embed_layer]
    y = batch[0]

    """
    # process y depends on model type
    # annotation classification
    if model.model_name == "LinearClassifier":
        le = LabelEncoder()
        y = le.fit_transform(y.flatten())
    """
    
    # ss3 / ss8 type
    if model.model_name == "MultiLabelMultiClass":
        # convert the y into np.arrays with -1 padding to the same length
        y = np.stack(
            [
                np.pad(
                    i,
                    pad_width=(0, x.shape[1] - len(i)),
                    constant_values=-1,
                )
                for i in y
            ]
        )

    return x.to(device, non_blocking=True), y.to(device, non_blocking=True)


def run_epoch(
    model: nn.Module,
    loader: DataLoader,
    encoder_name: str,
    embed_layer: int,
    reset_param: bool = False,
    resample_param: bool = False,
    embed_batch_size: int = 0,
    flatten_emb: bool | str = False,
    # if_encode_all: bool = True,
    device: torch.device | str = DEVICE,
    criterion: nn.Module | None = None,
    optimizer: torch.optim.Optimizer | None = None,
    **encoder_params,
) -> float:

    """
    Runs one epoch.
    
    Args:
    - model: nn.Module, already moved to device
    - loader: torch.utils.data.DataLoader
    - device: torch.device or str
    - criterion: optional nn.Module, loss function, already moved to device
    - optimizer: optional torch.optim.Optimizer, must also provide criterion,
        only provided for training

    Returns: 
    - float, average loss over batches
    """
    if optimizer is not None:
        assert criterion is not None
        model.train()
        is_train = True
    else:
        model.eval()
        is_train = False

    cum_loss = 0.0

    with torch.set_grad_enabled(is_train):
        # if not if_encode_all:
        # for each batch: y, sequence, mut_name, mut_numb, [layer0, ...]
        for batch in loader:
            x, y = get_x_y(model, device, batch, embed_layer)

            """
            x = batch[4][embed_layer]
            y = batch[0]

            # process y depends on model type
            # annotation classification
            if model.model_name == "LinearClassifier":
                le = LabelEncoder()
                y = le.fit_transform(y.flatten())
            # ss3 / ss8 type
            elif model.model_name == "MultiLabelMultiClass":
                # convert the y into np.arrays with -1 padding to the same length
                y = np.stack(
                    [
                        np.pad(
                            np.array(i[1:-1].split(", ")),
                            pad_width=(0, x.shape[1] - len(i)),
                            constant_values=-1,
                        )
                        for i in y
                    ]
                )

            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            """

            outputs = model(x)

            if criterion is not None:
                if model.model_name == "LinearRegression":
                    loss = criterion(outputs, y.float())
                elif model.model_name == "LinearClassifier":
                    loss = criterion(outputs, y.squeeze())

                if optimizer is not None:
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                cum_loss += loss.item()

    return cum_loss / len(loader)


def train(
    model: nn.Module,
    criterion: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    encoder_name: str,
    embed_layer: int,
    reset_param: bool = False,
    resample_param: bool = False,
    embed_batch_size: int = 0,
    flatten_emb: bool | str = False,
    device: torch.device | str = DEVICE,
    learning_rate: float = 1e-4,
    lr_decay: float = 0.1,
    epochs: int = 100,
    early_stop: bool = True,
    tolerance: int = 10,
    min_epoch: int = 5,
    **encoder_params,
) -> tuple[np.ndarray, np.ndarray]:

    """
    Args:
    - model: nn.Module, already moved to device
    - train_loader: torch.utils.data.DataLoader, 
    - val_loader: torch.utils.data.DataLoader, 
    - criterion: nn.Module, loss function, already moved to device
    - device: torch.device or str
    - learning_rate: float
    - lr_decay: float, factor by which to decay LR on plateau
    - epochs: int, number of epochs to train for
    - early_stop: bool = True,

    Returns: 
    - tuple of np.ndarray, (train_losses, val_losses)
        train/val_losses: np.ndarray, shape [epochs], entries are average loss
        over batches for that epoch
    """

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", factor=lr_decay
    )

    train_losses = np.zeros(epochs)
    val_losses = np.zeros(epochs)

    # init for early stopping
    counter = 0
    min_val_loss = np.Inf

    for epoch in tqdm(range(epochs)):

        train_losses[epoch] = run_epoch(
            model=model,
            loader=train_loader,
            encoder_name=encoder_name,
            embed_layer=embed_layer,
            reset_param=reset_param,
            resample_param=resample_param,
            embed_batch_size=embed_batch_size,
            flatten_emb=flatten_emb,
            device=device,
            criterion=criterion,
            optimizer=optimizer,
            **encoder_params,
        )

        val_loss = run_epoch(
            model=model,
            loader=val_loader,
            encoder_name=encoder_name,
            embed_layer=embed_layer,
            reset_param=reset_param,
            resample_param=resample_param,
            embed_batch_size=embed_batch_size,
            flatten_emb=flatten_emb,
            device=device,
            criterion=criterion,
            optimizer=None,
            **encoder_params,
        )
        val_losses[epoch] = val_loss

        scheduler.step(val_loss)

        if early_stop:
            # when val loss decrease, reset min loss and counter
            if val_loss < min_val_loss:
                min_val_loss = val_loss
                counter = 0
            else:
                counter += 1

            if epoch > min_epoch and counter == tolerance:
                break

    return train_losses, val_losses


def test(
    model: nn.Module,
    loader: DataLoader,
    embed_layer: int,
    criterion: nn.Module | None,
    device: torch.device | str = DEVICE,
    print_every: int = 1000,
) -> tuple[float, np.ndarray, np.ndarray]:
    """
    Runs one epoch of testing, returning predictions and labels.
    
    Args:
    - model: nn.Module, already moved to device
    - device: torch.device or str
    - loader: torch.utils.data.DataLoader
    - criterion: optional nn.Module, loss function, already moved to device
    - print_every: int, how often (number of batches) to print avg loss
    
    Returns: tuple (avg_loss, preds, labels)
    - avg_loss: float, average loss per training example 
    - preds: np.ndarray, shape [num_examples, ...], predictions over dataset
    - labels: np.ndarray, shape [num_examples, ...], dataset labels
    """
    model.eval()
    msg = "[{step:5d}] loss: {loss:.3f}"

    cum_loss = 0.0

    pred_probs = []
    pred_classes = []
    labels = []

    with torch.no_grad():

        for i, batch in enumerate(tqdm(loader)):
            # for each batch: y, sequence, mut_name, mut_numb, [layer0, ...]
            x, y = get_x_y(model, device, batch, embed_layer)

            """
            x = batch[4][embed_layer]
            y = batch[0]

            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            """

            # forward + backward + optimize
            outputs = model(x)

            # append results
            labels.extend(y.detach().cpu().squeeze().numpy())

            # append class
            if model.model_name == "LinearClassifier":
                pred_classes.extend(
                    outputs.detach()
                    .cpu()
                    .data.max(1, keepdim=True)[1]
                    .squeeze()
                    .numpy()
                )
                pred_probs.append(outputs.detach().cpu().squeeze().numpy())
            else:
                pred_probs.extend(outputs.detach().cpu().squeeze().numpy())

            if criterion is not None:
                if model.model_name == "LinearRegression":
                    loss = criterion(outputs, y)
                elif model.model_name == "LinearClassifier":
                    loss = criterion(outputs, y.squeeze())
                cum_loss += loss.item()

                if ((i + 1) % print_every == 0) or (i + 1 == len(loader)):
                    tqdm.write(msg.format(step=i + 1, loss=cum_loss / len(loader)))

    avg_loss = cum_loss / len(loader)
    return avg_loss, pred_probs, pred_classes, labels

In [11]:
"""Script for running pytorch models"""

from __future__ import annotations

import os
from tqdm import tqdm
from concurrent import futures

import torch
import torch.nn as nn

from sklearn.metrics import ndcg_score
from scipy.stats import spearmanr

from scr.params.sys import RAND_SEED, DEVICE
from scr.params.emb import TRANSFORMER_INFO, CARP_INFO

from scr.preprocess.data_process import split_protrain_loader, DatasetInfo
from scr.encoding.encoding_classes import get_emb_info, ESMEncoder, CARPEncoder
from scr.model.pytorch_model import LinearRegression, LinearClassifier
# from scr.model.train_test import train, test
from scr.vis.learning_vis import plot_lc
from scr.utils import get_folder_file_names, pickle_save, get_default_output_path


class Run_Pytorch:
    def __init__(
        self,
        dataset_path: str,
        encoder_name: str,
        reset_param: bool = False,
        resample_param: bool = False,
        embed_batch_size: int = 128,
        flatten_emb: bool | str = False,
        embed_folder: str | None = None,
        seq_start_idx: bool | int = False,
        seq_end_idx: bool | int = False,
        loader_batch_size: int = 64,
        worker_seed: int = RAND_SEED,
        if_encode_all: bool = True,
        if_multiprocess: bool = False,
        learning_rate: float = 1e-4,
        lr_decay: float = 0.1,
        epochs: int = 100,
        early_stop: bool = True,
        tolerance: int = 10,
        min_epoch: int = 5,
        device: torch.device | str = DEVICE,
        all_plot_folder: str = "results/learning_curves",
        all_result_folder: str = "results/train_val_test",
        **encoder_params,
    ) -> None:

        """
        A function for running pytorch model

        Args:
        - dataset_path: str, full path to the dataset, in pkl or panda readable format
            columns include: sequence, target, set, validation, mut_name (optional), mut_numb (optional)
        - encoder_name: str, the name of the encoder
        
        - embed_batch_size: int, set to 0 to encode all in a single batch
        - flatten_emb: bool or str, if and how (one of ["max", "mean"]) to flatten the embedding
        - embed_folder: str = None, path to presaved embedding
        - seq_start_idx: bool | int = False, the index for the start of the sequence
        - seq_end_idx: bool | int = False, the index for the end of the sequence
        - loader_batch_size: int, the batch size for train, val, and test dataloader
        - worker_seed: int, the seed for dataloader
        - learning_rate: float
        - lr_decay: float, factor by which to decay LR on plateau
        - epochs: int, number of epochs to train for
        - device: torch.device or str
        - all_plot_folder: str, the parent folder path for saving all the learning curves
        - all_result_folder: str = "results/train_val_test", the parent folder for all results
        - encoder_params: kwarg, additional parameters for encoding

        Returns:
        - result_dict: dict, with the keys and dict values
            "losses": {"train_losses": np.ndarray, "val_losses": np.ndarray}
            "train": {"mse": float, 
                    "pred": np.ndarray,
                    "true": np.ndarray,
                    "ndcg": float,
                    "rho": SpearmanrResults(correlation=float, pvalue=float)}
            "val":   {"mse": float, 
                    "pred": np.ndarray,
                    "true": np.ndarray,
                    "ndcg": float,
                    "rho": SpearmanrResults(correlation=float, pvalue=float)}
            "test":  {"mse": float, 
                    "pred": np.ndarray,
                    "true": np.ndarray,
                    "ndcg": float,
                    "rho": SpearmanrResults(correlation=float, pvalue=float)}

        """

        self._dataset_path = dataset_path
        self._encoder_name = encoder_name
        self._reset_param = reset_param
        self._resample_param = resample_param
        self._embed_batch_size = embed_batch_size
        self._flatten_emb = flatten_emb

        self._learning_rate = learning_rate
        self._lr_decay = lr_decay
        self._epochs = epochs
        self._early_stop = early_stop
        self._tolerance = tolerance
        self._min_epoch = min_epoch
        self._device = device
        self._all_plot_folder = all_plot_folder
        self._all_result_folder = all_result_folder
        self._encoder_params = encoder_params

        self._ds_info = DatasetInfo(self._dataset_path)
        self._model_type = self._ds_info.model_type
        self._numb_class = self._ds_info.numb_class

        self._train_loader, self._val_loader, self._test_loader = split_protrain_loader(
            dataset_path=self._dataset_path,
            encoder_name=self._encoder_name,
            reset_param=self._reset_param,
            resample_param=self._resample_param,
            embed_batch_size=self._embed_batch_size,
            flatten_emb=self._flatten_emb,
            embed_folder=embed_folder,
            seq_start_idx=seq_start_idx,
            seq_end_idx=seq_end_idx,
            subset_list=["train", "val", "test"],
            loader_batch_size=loader_batch_size,
            worker_seed=worker_seed,
            if_encode_all=if_encode_all,
            **encoder_params,
        )

        encoder_name, encoder_class, total_emb_layer = get_emb_info(encoder_name)

        if encoder_class == ESMEncoder:
            self._encoder_info_dict = TRANSFORMER_INFO
        elif encoder_class == CARPEncoder:
            self._encoder_info_dict = CARP_INFO

        if if_multiprocess:
            print("Running different emb layer in parallel...")
            # add the thredpool max_workers=None
            with futures.ProcessPoolExecutor(max_workers=os.cpu_count() - 1) as pool:
                # for each layer train the model and save the model
                for embed_layer in tqdm(range(total_emb_layer)):
                    pool.submit(self.run_pytorch_layer, embed_layer)

        else:
            for embed_layer in range(total_emb_layer):
                print(f"Running pytorch model for layer {embed_layer}")
                self.run_pytorch_layer(embed_layer)

    def run_pytorch_layer(self, embed_layer):

        # init model based on datasets
        if self._model_type == "LinearRegression":
            model = LinearRegression(
                input_dim=self._encoder_info_dict[self._encoder_name][0], output_dim=1
            )
            criterion = nn.MSELoss()

        elif self._model_type == "LinearClassifier":
            model = LinearClassifier(
                input_dim=self._encoder_info_dict[self._encoder_name][0], 
                numb_class=self._numb_class
            )
            criterion = nn.CrossEntropyLoss()
        
        model.to(self._device, non_blocking=True)

        criterion.to(self._device, non_blocking=True)

        train_losses, val_losses = train(
            model=model,
            criterion=criterion,
            train_loader=self._train_loader,
            val_loader=self._val_loader,
            encoder_name=self._encoder_name,
            embed_layer=embed_layer,
            reset_param=self._reset_param,
            resample_param=self._resample_param,
            embed_batch_size=self._embed_batch_size,
            flatten_emb=self._flatten_emb,
            device=self._device,
            learning_rate=self._learning_rate,
            lr_decay=self._lr_decay,
            epochs=self._epochs,
            early_stop=self._early_stop,
            tolerance=self._tolerance,
            min_epoch=self._min_epoch,
            **self._encoder_params,
        )

        # record the losses
        result_dict = {
            "losses": {"train_losses": train_losses, "val_losses": val_losses}
        }

        plot_lc(
            train_losses=train_losses,
            val_losses=val_losses,
            dataset_path=self._dataset_path,
            encoder_name=self._encoder_name,
            embed_layer=embed_layer,
            flatten_emb=self._flatten_emb,
            all_plot_folder=self._all_plot_folder,
        )

        # now test the model with the test data
        for subset, loader in zip(
            ["train", "val", "test"],
            [self._train_loader, self._val_loader, self._test_loader],
        ):
            loss, pred, cls, true = test(
                model=model, loader=loader, embed_layer=embed_layer, device=self._device, criterion=criterion
            )

            if model.model_name == "LinearRegression":
                result_dict[subset] = {
                    "mse": loss,
                    "pred": pred,
                    "true": true,
                    "ndcg": ndcg_score(true[None, :], pred[None, :]),
                    "rho": spearmanr(true, pred),
                }

            elif model.model_name == "LinearClassifier":
                result_dict[subset] = {
                    "cross-entropy": loss,
                    "pred": pred,
                    "true": true,
                    "acc": accuracy_score(true, cls),
                    "rocauc": roc_auc_score(true, pred, multi_class="ovo"),
                }            

        dataset_subfolder, file_name = get_folder_file_names(
            parent_folder=get_default_output_path(self._all_result_folder),
            dataset_path=self._dataset_path,
            encoder_name=self._encoder_name,
            embed_layer=embed_layer,
            flatten_emb=self._flatten_emb,
        )

        print(f"Saving results for {file_name} to: {dataset_subfolder}...")
        pickle_save(
            what2save=result_dict,
            where2save=os.path.join(dataset_subfolder, file_name + ".pkl"),
        )

In [31]:
Run_Pytorch(
        dataset_path="data/annotation/scl/balanced.csv",
        encoder_name="esm1_t6_43M_UR50S",
        reset_param = False,
        resample_param = False,
        embed_batch_size = 128,
        flatten_emb = "mean",
        embed_folder= "embeddings/annotation/scl/balanced",
        seq_start_idx = False,
        seq_end_idx = False,
        loader_batch_size = 4,
        worker_seed= RAND_SEED,
        if_encode_all = False,
        learning_rate = 1e-4,
        lr_decay = 0.1,
        epochs = 2,
        early_stop = True,
        tolerance = 10,
        min_epoch = 5,
        device = DEVICE,
        all_plot_folder = "results/learning_curves",
        all_result_folder = "results/train_val_test",
        # **encoder_params
    )

Generating esm1_t6_43M_UR50S upto 6 layer embedding ...


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


Converting classes into int...
Generating esm1_t6_43M_UR50S upto 6 layer embedding ...


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


Converting classes into int...
Generating esm1_t6_43M_UR50S upto 6 layer embedding ...


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main
  0%|          | 0/2 [00:00<?, ?it/s]

Converting classes into int...
Running pytorch model for layer 0


  0%|          | 0/2376 [00:00<?, ?it/s]


Making results/learning_curves/annotation ...
Making results/learning_curves/annotation/scl ...
Making results/learning_curves/annotation/scl/balanced ...
Making results/learning_curves/annotation/scl/balanced/esm1_t6_43M_UR50S ...
Making results/learning_curves/annotation/scl/balanced/esm1_t6_43M_UR50S/mean ...


RuntimeError: 1D target tensor expected, multi-target not supported

In [26]:
Run_Pytorch(
        dataset_path="data/proeng/gb1/low_vs_high.csv",
        encoder_name="esm1_t6_43M_UR50S",
        reset_param = False,
        resample_param = False,
        embed_batch_size = 128,
        flatten_emb = "mean",
        embed_folder= "embeddings/proeng/gb1/low_vs_high",
        seq_start_idx = False,
        seq_end_idx = False,
        loader_batch_size = 64,
        worker_seed= RAND_SEED,
        if_encode_all = False,
        learning_rate = 1e-4,
        lr_decay = 0.1,
        epochs = 2,
        early_stop = True,
        tolerance = 10,
        min_epoch = 5,
        device = DEVICE,
        all_plot_folder = "test/learning_curves",
        all_result_folder = "test/train_val_test",
        # **encoder_params
    )

Generating esm1_t6_43M_UR50S upto 6 layer embedding ...


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


Generating esm1_t6_43M_UR50S upto 6 layer embedding ...


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


Generating esm1_t6_43M_UR50S upto 6 layer embedding ...


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main
  0%|          | 0/2 [00:00<?, ?it/s]

Running pytorch model for layer 0


  0%|          | 0/2 [00:00<?, ?it/s]


UnboundLocalError: local variable 'loss' referenced before assignment

In [9]:
Run_Pytorch(
        dataset_path="data/structure/secondary_structure/casp12.csv",
        encoder_name="esm1_t6_43M_UR50S",
        reset_param = False,
        resample_param = False,
        embed_batch_size = 128,
        flatten_emb = "mean",
        # embed_folder= "embeddings/annotation/scl/balanced",
        seq_start_idx = False,
        seq_end_idx = False,
        loader_batch_size = 64,
        worker_seed= RAND_SEED,
        if_encode_all = False,
        learning_rate = 1e-4,
        lr_decay = 0.1,
        epochs = 2,
        early_stop = True,
        tolerance = 10,
        min_epoch = 5,
        device = DEVICE,
        all_plot_folder = "results/learning_curves",
        all_result_folder = "results/train_val_test",
        # **encoder_params
    )

Generating esm1_t6_43M_UR50S upto 6 layer embedding ...


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


Converting ss3/ss8 into np.array...
Generating esm1_t6_43M_UR50S upto 6 layer embedding ...


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


Converting ss3/ss8 into np.array...
Generating esm1_t6_43M_UR50S upto 6 layer embedding ...


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


Converting ss3/ss8 into np.array...
Running pytorch model for layer 0


UnboundLocalError: local variable 'model' referenced before assignment